# Sentiment Classification


## Loading the dataset

In [1]:
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers import LSTM 
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
from keras.datasets import imdb
import numpy as np
vocab_size = 10000 #vocab size
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

# restore np.load for future normal usage
np.load = np_load_old

In [0]:
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [4]:
import keras
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset
word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in x_train[0] ))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [5]:
from numpy import array
from keras.preprocessing.text import one_hot
docs = ['Gut gemacht',
		'Gute arbeit',
		'Super idee',
		'Perfekt erledigt',
		'exzellent',
		'naja',
		'Schwache arbeit.',
		'Nicht gut',
		'Miese arbeit.',
		'Hätte es besser machen können.']
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[41, 6], [1, 24], [1, 38], [1, 26], [14], [39], [28, 24], [42, 41], [4, 24], [34, 14, 24, 45, 8]]


## Train test split

In [0]:
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [7]:
print(len(x_train))
len(x_test)

25000


25000

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [8]:
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=maxlen)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 

W0104 17:24:51.198188 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0104 17:24:51.201338 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0104 17:24:51.204381 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0104 17:24:51.400079 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0104 17:24:51.414638 140594810365824 module_wrapper.py:139] From /usr/local

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 32)           1600      
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 54,901
Trainable params: 54,901
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

W0104 17:24:51.965445 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0104 17:24:52.029702 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0104 17:24:52.077456 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0104 17:24:52.081507 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0104 17:24:52.082561 140594810365824 module_wrapper.py:139] From /usr/local/li

Train on 25000 samples, validate on 25000 samples
Epoch 1/3


W0104 17:24:52.296036 140594810365824 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



25000/25000 [==============================] - 219s 9ms/step - loss: 0.6745 - acc: 0.5832 - val_loss: 0.6453 - val_acc: 0.6303
Epoch 2/3
25000/25000 [==============================] - 217s 9ms/step - loss: 0.6501 - acc: 0.6238 - val_loss: 0.6412 - val_acc: 0.6346
Epoch 3/3
25000/25000 [==============================] - 219s 9ms/step - loss: 0.6488 - acc: 0.6241 - val_loss: 0.6404 - val_acc: 0.6365


In [0]:
# Final evaluation of the model 
scores = model.evaluate(x_test, y_test, verbose=0) 

In [15]:
print("Accuracy of the model is: %.2f%%" % (scores[1]*100))

Accuracy of the model is: 63.65%


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [13]:
bad = "did not like the movie at all"
good = "movie was great"
for review in [good,bad]:
    tmp = []
    for word in review.split(" "):
        tmp.append(word_to_id[word])
    tmp_padded = sequence.pad_sequences([tmp], maxlen=maxlen) 
    print("%s . Sentiment: %s" % (review,model.predict(array([tmp_padded][0]))[0][0]))

movie was great . Sentiment: 0.5450077
did not like the movie at all . Sentiment: 0.4069479
